In [2]:
import pandas as pd
import ms3 
from  ms3.utils import *
import os 
from ms3 import Score
import matplotlib.pyplot as plt

In [4]:
path_midi = '/home/nulpe/Desktop/Tresillo/dataset/project_midi/billboard/'
target = '/home/nulpe/Desktop/Tresillo/dataset/project_mscx/mscx_billboard/'

dir_list = os.listdir(path_midi)

for el in dir_list:
    convert(path_midi+el, target+el[:-4]+'.mscx', MS='musescore3')

In [5]:
dir_sheet_music = '/home/nulpe/Desktop/Tresillo/dataset/tresillo midi billboards/ms3/'
list_sheet_music = os.listdir(dir_sheet_music)

for idx, el in enumerate(list_sheet_music):
    #Get notes with onsets
    s = Score(dir_sheet_music+el)
    df = s.mscx.notes
    
    # Define instruments 
    instrument_dict = {}
    for key in s.mscx.metadata['parts']:
        for staff in s.mscx.metadata['parts'][key].keys():
            instrument_dict[staff] = key
            
    
    #staff to instruments 
    df['instrument'] = [instrument_dict[el] if el in instrument_dict else 'na' for el in df.staff]
    
    
    # define quarter beat 
    df['quarter_beats'] = (df.mc_onset*16).astype('int32')
    
    
    #make rythm matrix & data frame
    df_histogram = df.groupby(['instrument','quarter_beats'])['mn'].agg(['count'])
    df_histogram = df_histogram.groupby(level=0).apply(lambda x: x.reset_index(level = 0).drop(['instrument'],axis=1).reindex(list(range(0,16)),fill_value=0).T)
    df_histogram = df_histogram.reset_index()

    df_histogram.insert(loc=0, column='song_artist', value=el[:-5])
    
    #concat to big rythm vector df
    if idx == 0: df_rythm_vectors = df_histogram
    
    df_rythm_vectors = pd.concat([df_rythm_vectors,df_histogram], axis=0)

In [29]:
df_rythm_vectors.to_csv('/home/nulpe/Desktop/Tresillo/dataset/rythm_vectors_tresillio.csv', index = False)
df_rythm_vectors = pd.read_csv('/home/nulpe/Desktop/Tresillo/dataset/rythm_vectors_tresillio.csv')
#df_rythm_vectors = df_rythm_vectors.drop(['index'], axis=1)

df_rythm_vectors

,song_artist,instrument,level_1,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,"Eastside-benny blanco, Halsey, Khalid",Athmosphere,count,32,0,0,0,0,0,0,0,30,0,0,0,30,0,0,0
1,"Eastside-benny blanco, Halsey, Khalid",Bright Piano,count,64,0,0,0,0,0,0,0,32,0,0,0,60,0,0,0
2,"Eastside-benny blanco, Halsey, Khalid",Fingered Bass,count,16,0,0,16,16,0,0,0,15,0,0,15,15,0,7,0
3,"Eastside-benny blanco, Halsey, Khalid",Jazz Guitar,count,48,0,0,0,0,0,0,0,45,0,0,0,45,0,0,0
4,"Eastside-benny blanco, Halsey, Khalid",Nylon Gtr.,count,48,0,0,0,0,0,0,0,45,0,0,0,45,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83,"Cheap Thrills(feat. Sean Paul)-Sia, Sean Paul","Syn.Strings 1, Cheap Thrills",count,36,0,0,0,0,0,0,0,18,0,0,0,0,0,0,0
84,"Cheap Thrills(feat. Sean Paul)-Sia, Sean Paul",Synth Voice,count,12,0,0,0,6,0,6,0,12,0,0,0,6,0,0,0
85,"Cheap Thrills(feat. Sean Paul)-Sia, Sean Paul",Tenor Sax,count,18,2,11,4,11,0,9,0,17,5,4,2,11,3,7,4
86,"Cheap Thrills(feat. Sean Paul)-Sia, Sean Paul",Voice Oohs,count,1,0,0,0,0,0,10,0,11,0,10,0,10,0,0,0
